In [1]:
import pandas as pd
from pandarallel import pandarallel
from evaluate import load
import re

pandarallel.initialize()

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
df = pd.read_parquet('gs://scraped-news-article-data-null/test_predicted.parquet')
df.head()

,source,id,category,title,published,body,summary,summary_type,predicted
0,reuters,112266,China,Millions tested in Shanghai as China grapples ...,2022-07-07T21:01:00,"summarize in bullet points: SHANGHAI/BEIJING, ...",* Third day of mass testing in many Shanghai d...,BULLETS,\n* China reported 338 new COVID cases for Wed...
1,reuters,66543,Business,Ecigarettes and heated tobacco light up Imperi...,2022-05-17T08:37:00,"summarize in bullet points: LONDON, May 17 (Re...",* H1 adjusted net revenue up 0.3% at constant ...,BULLETS,"\n* Shares up more than 7% in morning trade, b..."
2,cnbc,7668,Sustainable Future,Private jet flights in Europe soar to record l...,2023-03-30T05:24:48+00:00,summarize in bullet points: A private jet is l...,* Analysis published Thursday by environmental...,BULLETS,\n* The number of private jet flights in Europ...
3,reuters,87382,Middle East,Israel's Netanyahu returns with hard-right cab...,2022-12-29T19:28:00,"summarize in bullet points: JERUSALEM, Dec 29 ...","* New government has nationalist, religious pa...",BULLETS,\n* Netanyahu's hard-right cabinet includes re...
4,reuters,15571,DiversityCorporate CounselCorporate Governance,Activist behind Harvard race case takes aim at...,2021-07-13T21:56:00,summarize in bullet points: (Reuters) - Edward...,* Lawsuit claims statutes discriminate against...,BULLETS,\n* The lawsuit seeks to invalidate laws requi...


In [3]:
sample = df.sample(10, random_state = 42)
sample

,source,id,category,title,published,body,summary,summary_type,predicted
2402,reuters,22071,Software company morphs into personal injury firm,Plaintiffs' BarLegal InnovationLegal IndustryL...,2022-06-27T22:46:00,summarize in bullet points: (Reuters) - A soft...,* Mighty Law says it will undercut personal in...,BULLETS,\n* A software company is opening its own law ...
1910,cnbc,8940,Personal Finance,College hopefuls have a new ultimate dream sch...,2023-03-15T13:11:19+00:00,summarize in bullet points: ## Massachusetts I...,"* This year, the school named by the highest n...",BULLETS,\n* A new survey of college-bound students and...
960,reuters,50327,LNGFuel OilGasHydrogenRenewable Fuels,Spain's Cepsa to invest 3 bln euros in green h...,2022-12-01T15:03:00,"summarize in bullet points: CADIZ, Spain, Dec ...","* To produce 300,000 tons of green hydrogen pe...",BULLETS,\n* Cepsa to invest 3 bln euros in renewable e...
2348,reuters,22786,U.S. Markets,"Wall St falls after recent strong gains, Alpha...",2023-02-09T00:30:00,"summarize in bullet points: NEW YORK, Feb 8 (R...",* Alphabet shares dive after Google AI chatbot...,BULLETS,"\n* S&P 500 futures down 1.2%, yuan up 0.2%, w..."
4087,cnbc,1864,Sports,Canadian billionaire Steve Apostolopoulos says...,2023-04-24T17:42:10+00:00,summarize in bullet points: Canadian billionai...,* Canadian businessman Steve Apostolopoulos sa...,BULLETS,\n* The saga over Washington's embattled footb...
3417,reuters,62215,Europe,Russia's Navalny has mystery ailment which may...,2023-04-13T14:39:00,"summarize in bullet points: MOSCOW, April 13 (...",* Navalny in acute pain with stomach complaint...,BULLETS,"\n* Alexei Navalny, Russia's most prominent op..."
3017,cnbc,5341,Sports,Fanatics CEO Michael Rubin selling ownership s...,2022-06-22T17:00:21+00:00,summarize in bullet points: ## Fanatics CEO an...,* Billionaire Michael Rubin is selling his own...,BULLETS,\n* Fanatics CEO Michael Rubin announced Wedne...
2045,cnbc,12506,Evolve,McDonald’s new battle over the way the Big Mac...,2023-05-13T14:00:01+00:00,summarize in bullet points: ## In this article...,* McDonald's Big Mac made the move to paper pa...,BULLETS,\n* McDonald's has made several big packaging ...
3123,cnbc,7953,Retail,Walmart sued by EEOC for firing deli worker wi...,2023-03-28T19:36:35+00:00,summarize in bullet points: The Walmart logo i...,* Federal authorities sued Walmart Monday for ...,BULLETS,\n* The U.S. Equal Employment Opportunity Comm...
2086,reuters,101132,Europe,"Britain, Germany in role reversal on AstraZene...",2021-05-07T08:21:00,"summarize in bullet points: BERLIN, May 7 (Reu...",* Britain restricts AstraZeneca to people aged...,BULLETS,\n* AstraZeneca's COVID-19 vaccine is now avai...


In [4]:
sample.to_excel('gs://scraped-news-article-data-null/summary_human_eval.xlsx')

In [5]:
df = pd.read_parquet('gs://scraped-news-article-data-null/test_predicted_og.parquet')
df.head()

,source,id,category,title,published,body,summary,summary_type,predicted
0,reuters,112266,China,Millions tested in Shanghai as China grapples ...,2022-07-07T21:01:00,"summarize in bullet points: SHANGHAI/BEIJING, ...",* Third day of mass testing in many Shanghai d...,BULLETS,China's economy has been hit by a series of ec...
1,reuters,66543,Business,Ecigarettes and heated tobacco light up Imperi...,2022-05-17T08:37:00,"summarize in bullet points: LONDON, May 17 (Re...",* H1 adjusted net revenue up 0.3% at constant ...,BULLETS,The company's shares were up 7% in early after...
2,cnbc,7668,Sustainable Future,Private jet flights in Europe soar to record l...,2023-03-30T05:24:48+00:00,summarize in bullet points: A private jet is l...,* Analysis published Thursday by environmental...,BULLETS,The number of private jet flights in Europe ha...
3,reuters,87382,Middle East,Israel's Netanyahu returns with hard-right cab...,2022-12-29T19:28:00,"summarize in bullet points: JERUSALEM, Dec 29 ...","* New government has nationalist, religious pa...",BULLETS,"Israel's new prime minister, Benjamin Netanyah..."
4,reuters,15571,DiversityCorporate CounselCorporate Governance,Activist behind Harvard race case takes aim at...,2021-07-13T21:56:00,summarize in bullet points: (Reuters) - Edward...,* Lawsuit claims statutes discriminate against...,BULLETS,The lawsuit seeks to invalidate laws requiring...


In [6]:
og_sample = df.loc[sample.index]
og_sample.head()

,source,id,category,title,published,body,summary,summary_type,predicted
2402,reuters,22071,Software company morphs into personal injury firm,Plaintiffs' BarLegal InnovationLegal IndustryL...,2022-06-27T22:46:00,summarize in bullet points: (Reuters) - A soft...,* Mighty Law says it will undercut personal in...,BULLETS,"The company, called Mighty Group Inc., is laun..."
1910,cnbc,8940,Personal Finance,College hopefuls have a new ultimate dream sch...,2023-03-15T13:11:19+00:00,summarize in bullet points: ## Massachusetts I...,"* This year, the school named by the highest n...",BULLETS,A new survey of college-bound students and the...
960,reuters,50327,LNGFuel OilGasHydrogenRenewable Fuels,Spain's Cepsa to invest 3 bln euros in green h...,2022-12-01T15:03:00,"summarize in bullet points: CADIZ, Spain, Dec ...","* To produce 300,000 tons of green hydrogen pe...",BULLETS,"The project will produce 300,000 tons of hydro..."
2348,reuters,22786,U.S. Markets,"Wall St falls after recent strong gains, Alpha...",2023-02-09T00:30:00,"summarize in bullet points: NEW YORK, Feb 8 (R...",* Alphabet shares dive after Google AI chatbot...,BULLETS,"U.S. stocks ended lower on Wednesday, weighed ..."
4087,cnbc,1864,Sports,Canadian billionaire Steve Apostolopoulos says...,2023-04-24T17:42:10+00:00,summarize in bullet points: Canadian billionai...,* Canadian businessman Steve Apostolopoulos sa...,BULLETS,The saga over Washington's embattled football ...


In [7]:
og_sample.to_excel('gs://scraped-news-article-data-null/summary_human_eval_og.xlsx')